In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)

In [2]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [3]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [4]:
def yunrip_data_prep():
    basedir = './국토교통부_실거래가_공개시스템/연립다세대/매매/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['전용면적단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
    
    concat_df['년'] = concat_df['계약날짜'].dt.year
    
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명']
    
    concat_df = concat_df[concat_df['년'] >= 2015]
    
    return concat_df[['지번주소'] + [col for col in concat_df.columns if col not in cols_to_drop]]

In [5]:
def officetel_data_prep():
    basedir = './국토교통부_실거래가_공개시스템/오피스텔/매매/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['전용면적단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
    
    concat_df['년'] = concat_df['계약날짜'].dt.year
    
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명']
    
    concat_df = concat_df[concat_df['년'] >= 2015]
        
    return concat_df[['지번주소'] + [col for col in concat_df.columns if col not in cols_to_drop]]

In [6]:
yunrip_df = yunrip_data_prep()
print(yunrip_df.shape)
yunrip_df.head()

16it [00:01, 10.74it/s]


(329098, 11)


,지번주소,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),년
358389,서울특별시 강남구 개포동 1216-4,(1216-4),33.77,21.67,24800,4,2012.0,3.0,2015-03-26,734.379627,2015
358390,서울특별시 강남구 개포동 1216-4,(1216-4),35.87,23.02,27200,4,2012.0,3.0,2015-06-23,758.293839,2015
358391,서울특별시 강남구 개포동 1216-4,(1216-4),29.12,18.68,22200,4,2012.0,3.0,2015-07-20,762.362637,2015
358392,서울특별시 강남구 개포동 1216-4,(1216-4),29.97,19.23,22500,3,2012.0,3.0,2015-08-06,750.750751,2015
358393,서울특별시 강남구 개포동 170-18,(170-18),26.60,21.53,21000,1,1988.0,27.0,2015-08-21,789.473684,2015


In [7]:
officetel_df = officetel_data_prep()
print(officetel_df.shape)
officetel_df.head()

16it [00:00, 41.66it/s]


(77955, 10)


,지번주소,단지명,전용면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),년
100156,서울특별시 강남구 개포동 13-3,대청타워,43.24,22000,14,1997.0,18.0,2015-01-08,508.788159,2015
100157,서울특별시 강남구 개포동 13-3,대청타워,32.44,15800,21,1997.0,18.0,2015-01-12,487.053021,2015
100158,서울특별시 강남구 개포동 13-3,대청타워,32.44,16000,10,1997.0,18.0,2015-01-19,493.218249,2015
100159,서울특별시 강남구 개포동 13-3,대청타워,32.44,15400,21,1997.0,18.0,2015-01-26,474.722565,2015
100160,서울특별시 강남구 개포동 13-3,대청타워,31.91,16000,26,1997.0,18.0,2015-01-28,501.410216,2015


In [8]:
land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8706295, 11)


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 100-0,2013,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,325000.0
1,서울특별시 강남구 개포동 100-0,2014,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,330000.0
2,서울특별시 강남구 개포동 100-0,2015,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,335000.0
3,서울특별시 강남구 개포동 100-0,2016,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,345000.0
4,서울특별시 강남구 개포동 100-0,2017,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,355000.0


In [ ]:
last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)
print(last_df.shape)
last_df.head()

In [ ]:
land_specs_df['도로접면'].value_counts()

In [ ]:
yunrip_merge_df = yunrip_df.merge(land_specs_df, on=['지번주소', '년'])
print(yunrip_merge_df.shape)
yunrip_merge_df.head()

In [ ]:
officetel_merge_df = officetel_df.merge(land_specs_df, on=['지번주소', '년'])
print(officetel_merge_df.shape)
officetel_merge_df.head()

In [ ]:
reference_df = pd.read_excel('./감정평가사_자료/이승준_총괄표.xlsx', header=[0,1]).dropna(subset=[('소재지','소재지')]).sort_values([('소재지', '지역'),('소재지', '소재지'), ('소재지', '지번')]).reset_index(drop=True)
print(reference_df.shape)
reference_df.head()

In [ ]:
reference_df[('소재지', '지번')] = reference_df[('소재지', '지번')].apply(landnum_modifier)

In [ ]:
reference_df['지번주소'] = '서울특별시 ' + reference_df[('소재지', '지역')] + ' ' + reference_df[('소재지', '소재지')]\
+ ' ' + reference_df[('소재지', '지번')]

In [ ]:
def date_modifier(x):
    # x is a string or datetime
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if '-' in x:
            x = x.replace('-', '.')
            
        splitted = x.split('.')
        
        if len(splitted) == 1:
            return np.nan
        else:
            return x

In [ ]:
reference_df['기준시점2'] = reference_df[('수입/비용', '기준시점')].apply(date_modifier)

In [ ]:
def get_year(x):
    if pd.isna(x) == True:
        return x
    else:
        return int(x.split('.')[0])

In [ ]:
reference_df['년'] = reference_df['기준시점2'].apply(get_year)

In [ ]:
reference_df['년'] = reference_df['년'].fillna(2020)
reference_df['년'].isna().sum()

In [ ]:
def get_bd_type(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if '오피' in x:
            return '오피스텔'
        else:
            return '다세대'

In [ ]:
reference_df['건물유형'] = reference_df[('건물내용', '건물용도')].apply(get_bd_type)

In [ ]:
reference_df = reference_df.merge(last_df, on=['지번주소']).reset_index(drop=True)

In [ ]:
reference_df.head()

In [ ]:
reference_df['최근3년50분위'] = np.nan
reference_df['최근3년50분위_평가단가와의격차'] = np.nan
reference_df['최근3년평균'] = np.nan
reference_df['최근3년평균_평가단가와의격차'] = np.nan
reference_df['최근2년50분위'] = np.nan
reference_df['최근2년50분위_평가단가와의격차'] = np.nan
reference_df['최근2년평균'] = np.nan
reference_df['최근2년평균_평가단가와의격차'] = np.nan
reference_df['당해50분위'] = np.nan
reference_df['당해50분위_평가단가와의격차'] = np.nan
reference_df['당해평균'] = np.nan
reference_df['당해평균_평가단가와의격차'] = np.nan

In [ ]:
for i in tqdm(range(reference_df.shape[0])):
    gu = reference_df.loc[i, ('소재지', '지역')].replace(' ', '')
    dong = reference_df.loc[i, ('소재지', '소재지')].replace(' ', '')
    
    
    land_purpose = reference_df.loc[i, ('토지내용', '용도지역')].replace(' ', '')[:3]
    year = int(reference_df.loc[i, '년'])
    bd_type = reference_df.loc[i, '건물유형'][0]
    doro_ = reference_df.loc[]
    
    if bd_type == '오피스텔':
        target_df = officetel_merge_df[(officetel_merge_df['지번주소'].str.contains(gu+' '+dong))&
                                       (officetel_merge_df['용도지역명1'].str.contains(land_purpose))&
                                       (officetel_merge_df['건물연식'] <= 5)
                                      ]
    else:
        target_df = yunrip_merge_df[(yunrip_merge_df['지번주소'].str.contains(gu+' '+dong))&
                                    (yunrip_merge_df['용도지역명1'].str.contains(land_purpose))&
                                    (yunrip_merge_df['건물연식'] <= 5)
                                   ]    
    
    
    target_year_minus_two_df = target_df[(target_df['년'] <= year)&(target_df['년'] >= year-2)]
    target_year_minus_one_df = target_df[(target_df['년'] <= year)&(target_df['년'] >= year-1)]
    target_year_df = target_df[target_df['년'] == year]
        
        
    reference_df.loc[i, '최근3년50분위'] = target_year_minus_two_df['전용면적단가(만원/㎡)'].median()
    reference_df.loc[i, '최근3년평균'] = target_year_minus_two_df['전용면적단가(만원/㎡)'].mean()
    reference_df.loc[i, '최근2년50분위'] = target_year_minus_one_df['전용면적단가(만원/㎡)'].median()
    reference_df.loc[i, '최근2년평균'] = target_year_minus_one_df['전용면적단가(만원/㎡)'].mean()
    reference_df.loc[i, '당해50분위'] = target_year_df['전용면적단가(만원/㎡)'].median()
    reference_df.loc[i, '당해평균'] = target_year_df['전용면적단가(만원/㎡)'].mean()

In [ ]:
cols_list = []
for i in range(reference_df.shape[1]):
    if i < 39:
        cols_list.append(reference_df.columns[i][0] + '_' + reference_df.columns[i][1])
    else:
        cols_list.append(reference_df.columns[i][0])

In [ ]:
reference_df.columns = cols_list

In [ ]:
cols = ['지번주소', '기준시점2', '건물내용_건물용도', '건물유형', '토지내용_용도지역', '토지내용_도로너비',
        '평가단가(원/전유㎡) _전유면적', '최근3년50분위', '최근3년50분위_평가단가와의격차', '최근3년평균',
        '최근3년평균_평가단가와의격차', '최근2년50분위', '최근2년50분위_평가단가와의격차', '최근2년평균',
        '최근2년평균_평가단가와의격차', '당해50분위', '당해50분위_평가단가와의격차', '당해평균', '당해평균_평가단가와의격차'
       ]
selected_df = reference_df[cols]

In [ ]:
selected_df['평가단가(원/전유㎡) _전유면적'] = selected_df['평가단가(원/전유㎡) _전유면적'] / 10000

In [ ]:
selected_cols = ['최근3년50분위', '최근3년평균', '최근2년50분위', '최근2년평균', '당해50분위', '당해평균']
for col in selected_cols:
    selected_df[col+'_평가단가와의격차'] = selected_df[col] / selected_df['평가단가(원/전유㎡) _전유면적']
    selected_df[col+'_오차'] = selected_df[col] - selected_df['평가단가(원/전유㎡) _전유면적']
    selected_df[col+'_오차절대값'] = np.abs(selected_df[col+'_오차'])
    selected_df[col+'_오차율'] = selected_df[col+'_오차'] / selected_df['평가단가(원/전유㎡) _전유면적'] * 100
    selected_df[col+'_절대값오차율'] = selected_df[col+'_오차절대값'] / selected_df['평가단가(원/전유㎡) _전유면적'] * 100

In [ ]:
base_cols = ['지번주소', '기준시점2', '건물내용_건물용도', '건물유형', '토지내용_용도지역', '토지내용_도로너비',
             '평가단가(원/전유㎡) _전유면적'
            ]
cols_to_show = []
for col in selected_cols:
    cols_to_show.append(col)
    cols_to_show.append(col+'_오차율')

In [ ]:
selected2_df = selected_df[(selected_df['건물내용_건물용도'].str.contains('다세'))|(selected_df['건물내용_건물용도'].str.contains('오피'))].dropna(subset=['기준시점2'])
selected2_df = selected2_df[base_cols + cols_to_show]
print(selected2_df.shape)
selected2_df

In [ ]:
inv_df = land_specs_df[land_specs_df['지번주소'] == '서울특별시 금천구 시흥동 169-30']
inv_df

In [ ]:
reference_df.head()

In [ ]:
reference_df.shape

In [ ]:
merge_df = reference_df.merge(land_specs_df, on=['지번주소', '년'], how='left')
print(merge_df.shape)
merge_df.head()

In [ ]:
nan_df = merge_df[merge_df['용도지역명1'].isna()]
print(nan_df.shape)

In [ ]:
nan_df

In [ ]:
inv_df = land_specs_df[land_specs_df['지번주소'].str.contains('우이동 23-8')]
print(inv_df.shape)

In [ ]:
inv_df

In [ ]:
inv_df = land_specs_df[land_specs_df['지번주소'].str.contains('구로구 개봉동 128-')]
print(inv_df.shape)

In [ ]:
inv_df['지번주소'].unique()

In [ ]:
land_specs_df = land_specs_df.sort_values(['지번주소', '년']).reset_index(drop=True)

In [ ]:
merge_df = reference_df.merge(last_df, on=['지번주소'])
print(merge_df.shape)
merge_df.head()